In [ ]:
%run main.py --data-directory train_data --exp_directory ROIExp --batch-size 2

Epoch 1/25
----------


 97%|█████████▋| 728/752 [12:45<00:08,  2.91it/s]

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Sat Mar 26 15:16:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GRID V100D-8C       On   | 00000000:00:05.0 Off |                    0 |
| N/A   N/A    P0    N/A /  N/A |    560MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!kill -9 11465

/bin/bash: line 0: kill: (11465) - No such process


In [12]:
!rm -r ./train_data/Images/.ipynb_checkpoints

rm: cannot remove './train_data/Images/.ipynb_checkpoints': No such file or directory


In [3]:
import torch

model = torch.load('./ROIExp/weights_original.pt')

In [2]:
import datahandler
dataloaders = datahandler.get_dataloader_single_folder(
        'train_data', batch_size=2)

In [3]:
dataloaders

{'Train': <torch.utils.data.dataloader.DataLoader at 0x2ac8ec2fc940>,
 'Test': <torch.utils.data.dataloader.DataLoader at 0x2ac8ec2fc880>}

In [4]:
import numpy as np
from tqdm import tqdm
from PIL import Image

model.eval()  # Set model to evaluate mode
preds = np.array([])
trues = np.array([])
device = 'cuda'
threshold = 0.5

# Iterate over data.
for sample in tqdm(iter(dataloaders['Test'])):
    inputs = sample['image'].to(device)
    masks = sample['mask'].to(device)
    #print(masks)
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        y_pred = outputs['out'].data.cpu().numpy()
        y_true = masks.data.cpu().numpy()
        preds = np.concatenate([preds, y_pred.ravel()])
        trues = np.concatenate([trues, y_true.ravel()])
        #print((y_pred>threshold).astype('uint8').shape)
        #print(sample['mask'][0])
        for i in range(y_pred.shape[0]): 
            pred_mask = Image.fromarray((y_pred>threshold)[i].astype('uint8').reshape((512, 512)))
            pred_mask.save("./pred_mask/" + sample['maskname'][i])

100%|██████████| 188/188 [01:28<00:00,  2.13it/s]


In [12]:
from sklearn.metrics import f1_score

f1_score(trues > 0, preds > 0.4)

7.373761008950909e-05

In [6]:
from sklearn.metrics import classification_report

classification_report(trues > 0, preds > 0.4, digits=4)

'              precision    recall  f1-score   support\n\n       False     0.3163    0.2608    0.2859  20119284\n        True     0.8180    0.8549    0.8361  78184716\n\n    accuracy                         0.7333  98304000\n   macro avg     0.5671    0.5579    0.5610  98304000\nweighted avg     0.7153    0.7333    0.7234  98304000\n'

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(trues.astype('uint8'), preds)

In [7]:
from collections import Counter

Counter(preds > 0.1)

Counter({True: 97747996, False: 556004})